## Trajectory Optimization with Differentiable Physics

### Imports

In [ ]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# math
import numpy as np
# optim
import torch
# physics
import warp as wp
import warp.sim as wps
import warp.sim.render as wpr
# graphics
from copy import deepcopy
import mitsuba as mi
from mitsuba.scalar_rgb import Transform4f as mit
# plots
import matplotlib as mpl
from matplotlib import pyplot as plt
from src.mpl_utils import set_fig_size, set_mpl_format

# setup
set_mpl_format()
wp.init()
mi.set_variant('llvm_ad_rgb')
print(f'Mitsuba 3 Variant: {mi.variant()}')

### Scene

In [ ]:
# camera positioned to view the scene, adjust as needed
cam_pos = np.array([0.0, 1.0, 5.0])  
cam_target = np.array([0.0, 1.0, 0.0])
cam_up = np.array([0.0, 1.0, 0.0])

# sphere
sphere_init_pos = np.array([-0.5, 1.0, 0.0])
sphere_init_vel = (15.0, -15.0, 0.0)
sphere_init_qd = (0.0, 0.0, 0.0) + sphere_init_vel
sphere_radius = 0.2
#sphere_mass = 1.0

target_sphere_mass = 1.0
sphere_mass = 1.5

sphere_density = sphere_mass / (4.0 / 3.0 * np.pi * sphere_radius**3)
sphere_rgb = np.array([0.2, 0.25, 0.7])

# wall
wall_pos = np.array([2.0, 1.0, 0.0])
wall_extents = np.array([0.25, 1.0, 1.0])
wall_rgb = np.array([0.7, 0.25, 0.2])


# target
#target_pos = [-2.0, 1.5, 0.0]
target_pos = [-1.6727347,  2.4678576, 0.0]
target_q = target_pos + [0.0, 0.0, 0.0, 1.0]

# contact parameters
ke = 1.0e4
kf = 0.0
kd = 1.0e1
mu = 0.2
rt = 1.0
margin = 10.0

# simulation parameters
sim_duration = 0.6
# control frequency
fps = 60
frame_dt = 1.0 / float(fps)
frame_steps = int(sim_duration / frame_dt)
# sim frequency
sim_substeps = 8
sim_steps = frame_steps * sim_substeps
sim_dt = frame_dt / sim_substeps

### Physics

In [ ]:
def build_phys(device):
    builder = wps.ModelBuilder()
    # particle
    #builder.add_particle(pos=sphere_init_pos, vel=sphere_init_vel, radius=sphere_radius, mass=sphere_mass)
    sphere_body_idx = builder.add_body(origin=wp.transform(sphere_init_pos, wp.quat_identity()))
    builder.add_shape_sphere(body=sphere_body_idx, 
                             radius=sphere_radius, 
                             density=sphere_density,
                             ke=ke, 
                             kf=kf, 
                             kd=kd, 
                             mu=mu,
                             restitution=rt)
    builder.body_qd[sphere_body_idx] = sphere_init_qd
    # wall
    builder.add_shape_box(body=-1, 
                          pos=wall_pos, 
                          hx=wall_extents[0], hy=wall_extents[1], hz=wall_extents[2], 
                          ke=ke, kf=kf, kd=kd, mu=mu)
    model = builder.finalize(device, requires_grad=True)

    # ground
    model.soft_contact_kf = kf
    model.soft_contact_margin = margin
    model.rigid_contact_margin = margin
    
    # allocate sim states for trajectory
    states = [model.state(requires_grad=True) for _ in range(sim_steps + 1)]

    return model, states


def capture_phys(device, model, integrator, states):
    # tape tracks computation graph
    tape = wp.Tape()
    wp.capture_begin(device=device)
    with tape:
        for i in range(sim_steps):
            states[i].clear_forces()
            wp.sim.collide(model, states[i])
            integrator.simulate(model, states[i], states[i + 1], sim_dt)
    graph = wp.capture_end()

    return graph, tape


def get_trajectory(states):
    trajectory = []
    for state in states[::sim_substeps]:
        trajectory.append(state.body_q.numpy()[0, :3])
    return np.array(trajectory)

### Rendering

In [ ]:
img_width = 256
img_height = 256
fov = 60

render_integrator = {
    'type': 'direct_reparam',
}

scene_dict = {
    'type': 'scene',
    'integrator': render_integrator,
    'sensor':  {
        'type': 'perspective',
        'to_world': mit.look_at(
                        origin=cam_pos,
                        target=cam_target,
                        up=cam_up),
        'fov': fov,
        'film': {
            'type': 'hdrfilm',
            'width': img_width,
            'height': img_height,
            'rfilter': { 'type': 'gaussian' },
            'sample_border': True
        },
    },
    "light": {
        "type": "constant"
    },
    'particle': {
        'type': 'sphere',
        'center': sphere_init_pos,
        'radius': sphere_radius,
        'bsdf': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': sphere_rgb
            }
        }
    },
    'wall': {
        'type': 'cube',
        'to_world': mit.translate(wall_pos).scale(wall_extents),
        'bsdf' : {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': wall_rgb
            }
        }
    },
    "ground": {
        "type": "rectangle",
        "to_world": mit.rotate(axis=[1.0, 0.0, 0.0], angle=-90).scale(20),
        "bsdf": {
            "type": "diffuse",
            'reflectance': {
                'type': 'checkerboard',
                'to_uv': mi.ScalarTransform4f.scale([10, 10, 1])
            }
        }
    }
}

### States

In [ ]:
# initial run 
phys_device = wp.get_cuda_devices()[0]
phys_model, phys_states = build_phys(phys_device)
phys_integrator = wp.sim.SemiImplicitIntegrator()
phys_graph, phys_tape = capture_phys(phys_device, phys_model, phys_integrator, phys_states)
wp.capture_launch(phys_graph)

pred_state = phys_states[-1].body_q.numpy()[0, :3]

In [ ]:
fig_size = set_fig_size(subplots=(1, 3))
fig, axs = plt.subplots(1, 3, figsize=fig_size, constrained_layout=True)

init_start_scene_dict = dict(scene_dict)
init_start_scene_dict['particle']['center'] = deepcopy(sphere_init_pos)
bitmap_init = mi.util.convert_to_bitmap( mi.render(mi.load_dict(init_start_scene_dict), spp=16))
axs[0].imshow(bitmap_init)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].set_title('Start State')


init_end_scene_dict = dict(scene_dict)
init_end_scene_dict['particle']['center'] = deepcopy(pred_state)
bitmap_init = mi.util.convert_to_bitmap( mi.render(mi.load_dict(init_end_scene_dict), spp=16))
axs[1].imshow(bitmap_init)
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_title('Initial End State')


target_scene_dict = dict(scene_dict)
target_scene_dict['particle']['center'] = deepcopy(target_pos)
bitmap_target = mi.util.convert_to_bitmap(mi.render(mi.load_dict(target_scene_dict), spp=16))
axs[2].imshow(bitmap_target)
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].set_title('Target End State')

plt.show()
#plt.savefig('fig_direct_traj_states_sphere.pdf', bbox_inches='tight')

### Optimization

In [ ]:
# training details
train_iters = 20
train_rate = 0.2
train_save_interval = 10

# physics
phys_device = wp.get_cuda_devices()[0]
phys_model, phys_states = build_phys(phys_device)
phys_integrator = wp.sim.SemiImplicitIntegrator()
phys_graph, phys_tape = capture_phys(phys_device, phys_model, phys_integrator, phys_states)

# variables
x = wp.to_torch(phys_model.body_inv_mass)
pred = wp.to_torch(phys_states[-1].body_q)
target = torch.tensor(target_q, dtype=torch.float32, device=torch.device('cuda')).unsqueeze(0)

# optimization
opt = torch.optim.Adam([x], lr=train_rate)
loss = torch.nn.MSELoss(reduction='sum')

# metrics
trajs = []
losses = []

for i in range(train_iters+1):
    # zero grads
    opt.zero_grad(set_to_none=False)
    phys_tape.zero()

    # forward pass
    wp.capture_launch(phys_graph)
    target.data[0, 3:] = pred.data[0, 3:]
    output = loss(pred, target)

    if i < train_iters:  
        # backprop
        output.backward()
        phys_pred = wp.from_torch(pred)
        phys_tape.backward(grads={phys_pred : phys_pred.grad})
        opt.step()
        torch.clamp(x, min=0.1, max=1.0)
        #x.data[0, :3] = 0.0
        #x.data[0, -1] = 0.0

    losses.append(output.cpu().detach().numpy())
    if i % train_save_interval == 0:
        print(f"iter_{i:02d}:\n \
                pose_loss={losses[-1]:6f},\n \
                mass={x.cpu().tolist()},\n \
                mass_grad={x.grad.cpu().tolist()},\n \
                end_pose=[{pred[0,0]:.4f}, {pred[0,1]:.4f}]")
        trajs.append(get_trajectory(phys_states))

### Results

In [ ]:
fig_size = set_fig_size(fraction=1.0, subplots=(1, 2))
fig, axs = plt.subplots(1, 2, figsize=fig_size, constrained_layout=True)

################
# Trajectories #
################

for i, vel_traj in enumerate(trajs):
    axs[0].plot(vel_traj[:, 0], vel_traj[:, 1], label=f'{i*train_save_interval}')
axs[0].scatter(sphere_init_pos[0], sphere_init_pos[1], facecolors='none', edgecolors='black', s=50)
axs[0].scatter(target_pos[0], target_pos[1], color='black', marker='+', s=50)
#axs[0].set_xlim([-2.1, 2.1])
#axs[0].set_yticks(np.arange(0.0, 2.6, 0.5))
#axs[0].set_ylim([0.0, 2.5])
axs[0].set_title('Trajectories') 
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')
axs[0].legend(handlelength=1.0, loc='upper right')

##########
# Losses #
##########

axs[1].plot(np.log(losses))
#axs[1].set_yticks(np.arange(0, 1.1, 0.5))
#axs[1].set_ylim([0, 1.1])
axs[1].set_title('Log Pose Loss per Iteration')
axs[1].set_xlabel('Iteration')
axs[1].set_ylabel('Log Pose Loss')

plt.show()
#plt.savefig('fig_direct_traj_optim_sphere.pdf', bbox_inches='tight')